In [1]:
from algo_reasoning.src.data.data import load_dataset, collate
from torch.utils.data import DataLoader
from algo_reasoning.src.models.encoder import Encoder

2024-05-27 12:24:13.170001: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ronald/Documents/env/algo_reasoning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
ds =  load_dataset("dag_shortest_paths", "train", "tmp/CLRS30")
loader = DataLoader(ds, batch_size=32, collate_fn=collate)

obj = next(iter(loader))

2024-05-27 12:26:09.527350: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".
2024-05-27 12:26:10.058 | INFO     | algo_reasoning.src.data.data:load_dataset:116 - Downloading dataset for algorithm 'dag_shortest_paths'...


Dataset clrs_dataset downloaded and prepared to tmp/CLRS30/clrs_dataset/dag_shortest_paths_train/1.0.0. Subsequent calls will reuse this data.


2024-05-27 12:26:38.966168: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [19]:
obj

CLRSDataBatch(inputs=CLRSDataBatch(A=[32, 16, 16], adj=[32, 16, 16], pos=[32, 16], s=[32, 16], batch=[32], ptr=[33]), hints=CLRSDataBatch(color=[32, 49, 16, 3], d=[32, 49, 16], mark=[32, 49, 16], phase=[32, 49], pi_h=[32, 49, 16], s_last=[32, 49, 16], s_prev=[32, 49, 16], topo_h=[32, 49, 16], topo_head_h=[32, 49, 16], u=[32, 49, 16], v=[32, 49, 16]), length=[32], outputs=CLRSDataBatch(pi=[32, 16]), algorithm='dag_shortest_paths')

In [20]:
encoder = Encoder("dag_shortest_paths")

result =  encoder(obj, hint_step=0)
result

2024-05-27 12:28:03.036 | DEBUG    | algo_reasoning.src.models.encoder:encode_CLRSData:75 - Encoding A.
2024-05-27 12:28:03.041 | DEBUG    | algo_reasoning.src.models.encoder:encode_CLRSData:75 - Encoding adj.
2024-05-27 12:28:03.049 | DEBUG    | algo_reasoning.src.models.encoder:encode_CLRSData:75 - Encoding pos.
2024-05-27 12:28:03.053 | DEBUG    | algo_reasoning.src.models.encoder:encode_CLRSData:75 - Encoding s.
2024-05-27 12:28:03.057 | WARNING  | algo_reasoning.src.models.encoder:encode_CLRSData:68 - Key batch not in specs for algorithm dag_shortest_paths
2024-05-27 12:28:03.058 | WARNING  | algo_reasoning.src.models.encoder:encode_CLRSData:68 - Key ptr not in specs for algorithm dag_shortest_paths
2024-05-27 12:28:03.060 | DEBUG    | algo_reasoning.src.models.encoder:encode_CLRSData:75 - Encoding color.
2024-05-27 12:28:03.062 | DEBUG    | algo_reasoning.src.models.encoder:encode_CLRSData:75 - Encoding d.
2024-05-27 12:28:03.066 | DEBUG    | algo_reasoning.src.models.encoder:enc

Key: A | Encoding: torch.Size([32, 16, 16, 128])
Key: adj | Encoding: torch.Size([32, 16, 16, 128])
Key: pos | Encoding: torch.Size([32, 16, 128])
Key: s | Encoding: torch.Size([32, 16, 128])
Key: color | Encoding: torch.Size([32, 16, 128])
Key: d | Encoding: torch.Size([32, 16, 128])
Key: mark | Encoding: torch.Size([32, 16, 128])
Key: phase | Encoding: torch.Size([32, 128])
Key: pi_h | Encoding: torch.Size([32, 16, 128])
Key: s_last | Encoding: torch.Size([32, 16, 128])
Key: s_prev | Encoding: torch.Size([32, 16, 128])
Key: topo_h | Encoding: torch.Size([32, 16, 128])
Key: topo_head_h | Encoding: torch.Size([32, 16, 128])
Key: u | Encoding: torch.Size([32, 16, 128])
Key: v | Encoding: torch.Size([32, 16, 128])


(tensor([[[-4.5402, -2.2897,  4.1059,  ...,  0.2397,  4.4627, -0.6933],
          [-3.5752, -1.6385,  2.7017,  ...,  0.6415,  3.4819, -1.1295],
          [-3.0983, -1.3094,  2.7280,  ...,  1.1737,  3.8037, -1.2646],
          ...,
          [-3.5502, -1.0907,  2.9353,  ...,  0.7410,  3.9674, -0.6990],
          [-3.3004, -0.7829,  2.9904,  ...,  0.7230,  3.3390, -0.1647],
          [-3.3713, -1.2622,  2.8686,  ...,  0.6364,  3.0894, -0.4264]],
 
         [[-4.5402, -2.2897,  4.1059,  ...,  0.2397,  4.4627, -0.6933],
          [-3.5752, -1.6385,  2.7017,  ...,  0.6415,  3.4819, -1.1295],
          [-3.0983, -1.3094,  2.7280,  ...,  1.1737,  3.8037, -1.2646],
          ...,
          [-3.5502, -1.0907,  2.9353,  ...,  0.7410,  3.9674, -0.6990],
          [-3.3004, -0.7829,  2.9904,  ...,  0.7230,  3.3390, -0.1647],
          [-3.3713, -1.2622,  2.8686,  ...,  0.6364,  3.0894, -0.4264]],
 
         [[-4.5402, -2.2897,  4.1059,  ...,  0.2397,  4.4627, -0.6933],
          [-3.5752, -1.6385,

In [21]:
node_fts, edge_fts, graph_fts = result

In [22]:
node_fts.shape

torch.Size([32, 16, 128])

In [23]:
edge_fts.shape

torch.Size([32, 16, 16, 128])

In [24]:
graph_fts.shape

torch.Size([32, 128])